In [24]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

from PIL import ImageEnhance
from PIL import ImageOps
import matplotlib.patches as patches


In [3]:
input_folder = 'myMNIST/'

digits_raw = []
labels = []

for i in range(10):
    filename = input_folder + str(i) + '.png'
    image = Image.open(filename)
    image = image.convert(mode = 'L')
    digits_raw.append(image)
    labels.append(i)

In [4]:
digits_centered = []

for digit in digits_raw:

    # Inver sample, get bbox and display all that stuff.
    inv_sample = ImageOps.invert(digit)
    bbox = inv_sample.getbbox()
    rect = patches.Rectangle(
        (bbox[0], bbox[3]), bbox[2]-bbox[0], -bbox[3]+bbox[1]-1,
        fill=False, alpha=1, edgecolor='w')

    crop = inv_sample.crop(bbox)

    new_size = 140
    delta_w = new_size - crop.size[0]
    delta_h = new_size - crop.size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    new_im = ImageOps.expand(crop, padding)
    
    digits_centered.append(new_im)

In [5]:
def normalize(img_in):
    img = np.asarray(img_in)
    
    # normalize
    max_val = float(img.max())
    #print(max_val)

    x_dim, y_dim = img.shape

    norm_img = np.zeros_like(img)

    for i in range(x_dim):
        for j in range(y_dim):
            norm_img[i, j] = img[i, j] / max_val
    
    return norm_img
    

In [6]:
digits_norm = []

for digit in digits_centered:
    digits_norm.append(normalize(digit))

In [7]:
def downscale(img_source, x0_raw, x1_raw):
    x_dim, y_dim = img_source.shape
    x0 = np.floor(x0_raw * x_dim*0.9999).astype(int)
    x1 = np.floor(x1_raw * y_dim*0.9999).astype(int)
    return img_source[x0, x1]

In [23]:
my_digits = []

# sample = digits_norm[9]

grid_size = 28
extent = [0, 1, 0, 1]

X = np.meshgrid(np.linspace(extent[0],extent[1],grid_size,dtype=float),
                np.linspace(extent[2],extent[3],grid_size,dtype=float), indexing='ij')

for digit in digits_norm:
    result = downscale(digit, *X)
    my_digits.append(result)